# WPROWADZENIE DO SZTUCZNEJ INTELIGENCJI - LABORATORIUM 6

+ AUTORZY: **MATEUSZ SZCZEPANOWSKI** i **ŁUKASZ STANISZEWSKI**
+ NUMERY INDEKSÓW: **298988** i **304098**
+ ADRESY E-MAIL: mateusz.szczepanowski.stud@pw.edu.pl i lukasz.staniszewski.stud@pw.edu.pl
+ KIERUNEK: **INFORMATYKA**
+ PRZEDMIOT: **Wprowadzenie do sztucznej inteligencji**
+ ZADANIE: **[LINK](https://apps.usos.pw.edu.pl/apps/f/mBHyr3Rd/lab6.pdf)**
+ SYSTEM OPERACYJNY: **Windows 10**
+ JĘZYK PROGRAMOWANIA: **Python 3.8**
+ TEMAT: **Należy zaimplementować perceptron wielowarstowowy oraz metodę uczącą go przy pomocy algorytmu propagacji wstecznej.Wykorzystując napisaną sieć neuronową należy wytrenować klasyfikator na
zbiorze danych MNIST i zbadać jego działanie.**

## 1. Import niezbędnych modułów
+ **keras.datasets.mnist** - w celu załadowania zbioru mnist
+ **numpy** - do operacji na macierzach
+ **math** - operacje matematyczne

In [1]:
from keras.datasets import mnist
import numpy as np
import math

## 2. ZBIÓR MNIST
+ Zbiór MNIST jest zbiorem 60000 obrazów uczących i 10000 testujących o rozmiarze 28x28 zawierających odręcznie zapisane cyfry 0-9 w skali szarości.
+ W celu operacji na zbiorze MNIST powstała funkcja GENERATE_MNIST_DATASETS() zwracająca próbki danych podzielonych na zbiór trenujący (50000), walidacyjny (10000) oraz testowy (10000). 

In [2]:
def generate_mnist_datasets(is_validation_needed=True, how_many_train=50000):
    # getting sets of data and reshaping them
    (X_trainvalid, Y_trainvalid), (X_test, Y_test) = mnist.load_data()
    # to get inputs as (754,1) length vector
    X_trainvalid = X_trainvalid.reshape(X_trainvalid.shape[0], 1, 28*28)
    X_trainvalid = X_trainvalid.astype('float32')
    # div to get (0-1)
    X_trainvalid /= 255
    # same for test
    X_test = X_test.reshape(X_test.shape[0], 1, 28*28)
    X_test = X_test.astype('float32')
    X_test /= 255
    # if we want also validation_set
    if is_validation_needed:
        X_train = X_trainvalid[:how_many_train]
        Y_train = Y_trainvalid[:how_many_train]
        X_valid = X_trainvalid[how_many_train:]
        Y_valid = Y_trainvalid[how_many_train:]
        return (X_train, Y_train), (X_valid, Y_valid), (X_test, Y_test)
    else:
        return (X_trainvalid, Y_trainvalid), (X_test, Y_test)

## 3. Zdefiniowanie problemu
### 3.1. Ogólnie
+ Zadanie polega na zaimplementowaniu perceptronu wielowarstwowego, zostanie on zaimplementowany w formie klasy Network, przyjmującej w konstruktorze listę liczb odpowiadającą ilościom neuronów w poszczególnych wartstwach perceptronu.

+ Dla klasy tej zostanie również zaimplementowana funkcja TRAIN_NETWORK() przyjmująca jako parametr zbiór trenujący model, a także parametr beta oraz minibatch_size (używany do stochastycznego spadku gradientu). Funkcja ta wykorzysta metodę spadku gradientu, do obliczenia gradientu wektora na podstawie gradientów podzbioru i użyje go w celu wykonania kroku GD. Pojedynczy gradient będzie liczony z wykorzystaniem wstecznej propagacji.

+ Na końcu nastąpi walidacja hiperparametru (liczby neuronów ukrytych w modelu) i wytrenowanie go na najlepszym możliwie hiperparametrze.

+ Na końcu zostanie przeprowadzone sprawdzenie działania sieci na zbiorze testowym.

### 3.2. Klasa Network
+ Klasa network jest obiektem reprezentującym sieć neuronową, w konstruktorze podawane są parametry sieci w postaci listy, gdzie kolejne liczby w liście to ilości neuronów w poszczególnych warstwach.
    + np. perceptron o warstwie wejściowej składającej się ze 100 wejść, 5 wyjść i 3 warstw ukrytych o  wielkościach 50, 30 i 10 otrzymuje na wejśćiu listę $ [100, 50, 30, 10, 5] $
+ Klasa posiada takie pola jak:
    + wielkości wartstw (layers_sizes)
    + liczba warstw (number_of_layers)
    + listę wektorów bias-ów neuronów dla poszczególnych warstw (biases)
    + listę macierzy wag neuronów dla poszczególnych warstw (weights)
    + liczbę klas do sklasyfikowania przez sieć (n_of_classess)
+ Metody klasy:
    + obliczanie wektoru klasyfikacji (forward_propagation)
    + wykonanie wstecznej propagacji na otrzymanej próbce (backward_propagation)
    + metoda gradientu prostego do optymalizacji (train_network)
    + metoda oceniająca sieć na otrzymanym zbiorze testowym (score_network)

In [3]:
class Network(object):
    def __init__(self, layers_sizes):
        self.layers_sizes = layers_sizes
        self.number_of_layers = len(layers_sizes)
        # initialization biases of neurons are from normal distribution
        self.biases = [np.random.randn(y, 1) for y in layers_sizes[1:]]
        # initialization weights of neurons are ~U(-1/sqrt(dim(input)),1/sqrt(dim(input)))
        self.weights = [np.random.uniform(low=-1 / math.sqrt(layers_sizes[0]), high=1 / math.sqrt(layers_sizes[0]), size=(y, x)) for x, y in zip(layers_sizes[:-1], layers_sizes[1:])]
        self.n_of_classess = layers_sizes[-1]
        
    def forward_propagation(self, z):
        # returned z is array with indexes of classes and values between 0 and 1 (max will be chosen) and index is class
        for b, w in zip(self.biases, self.weights):
            if w.shape[1] != z.shape[0]:
                z = z.transpose()
            z = sigmoid(np.dot(w, z) + b)
        return z

    def backward_propagation(self, vector_input, values):
        # first we need matrixes of neuron's weights and biases to be zeros
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        # input should be vertical vector
        vector_input = vector_input.transpose()
        # first activation layer is vector of inputs
        activations = [vector_input]
        # current_activation
        curr_activation = vector_input
        # generating list of layers of z's for neurons
        z_list = []
        # for each layer of neurons
        for b, w in zip(self.biases, self.weights):
            # getting list of z's for layer of neurons
            z = np.dot(w, curr_activation) + b
            # appending it to list
            z_list.append(z)
            # new activation is sigmoid(before_activation)
            curr_activation = sigmoid(z)
            # and adding as new layer of activation
            activations.append(curr_activation)
        # important case: different complex derivatives for last layer (output layer) 
        # ∂C/∂zL = ∂C/∂aL x ∂aL/∂zL
        deriv_cost_z = 2 * (activations[-1] - values) * sigmoid_derivative(z_list[-1])
        # ∂C/∂bL = 1 x ∂C/∂zL
        grad_b[-1] = deriv_cost_z
        # ∂C/∂wL = ∂C/∂zL x aL-1
        grad_w[-1] = np.dot(deriv_cost_z, activations[-2].transpose())
        # going backward from second-last layer to first
        for layer in range(2, self.number_of_layers):
            curr_z = z_list[-layer]
            # ∂C/∂zK = ∂zK+1/∂aK x ∂C/∂zK+1 x ∂aK/∂zK
            deriv_cost_z = np.dot(self.weights[-layer + 1].transpose(), deriv_cost_z) * sigmoid_derivative(curr_z)
            # ∂C/∂bK = 1 x ∂C/∂zK
            grad_b[-layer] = deriv_cost_z
            # ∂C/∂wL = ∂C/∂zK x aK-1
            grad_w[-layer] = np.dot(deriv_cost_z, activations[-layer - 1].transpose())
        # retuning gradients for biases and weights of neurons
        return (grad_b, grad_w)

    def train_network(self, training_data, beta, evolutions=5):
        # taking inputs and values
        training_data_inputs = training_data[0]
        training_data_values = training_data[1]
        for _ in range(evolutions):
            # taking x as vector of inputs to network and number as class
            for x, y in zip(training_data_inputs, training_data_values):
                # zeros at start
                gradients_b = [np.zeros(b.shape) for b in self.biases]
                gradients_w = [np.zeros(w.shape) for w in self.weights]
                # instead of single value, y should be list of n elements with values 0.0 or 1.0
                # ex: for y=7 and n_of_classes = 10, y_list = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
                y_list = np.zeros(shape=(self.n_of_classess,1))
                y_list[y] = 1.0
                # getting gradients for each layer
                deriv_gradients_b, deriv_gradients_w = self.backward_propagation(x, y_list)
                # summing up gradients
                gradients_b = [grad_b + d_grad_b for grad_b, d_grad_b in zip(gradients_b, deriv_gradients_b)]
                gradients_w = [grad_w + d_grad_w for grad_w, d_grad_w in zip(gradients_w, deriv_gradients_w)]
                # setting up weights and biases of network
                self.weights = [weight - beta * grad_w for weight, grad_w in zip(self.weights, gradients_w)]
                self.biases = [bias - beta * grad_b for bias, grad_b in zip(self.biases, gradients_b)]

    def score_network(self, data_set):
        # taking inputs and outputs
        data_inputs = data_set[0]
        data_outputs = data_set[1]
        # how many correct
        result = 0
        # for every input, output
        for x, y in zip(data_inputs, data_outputs):
            # classify input as creating array (10,1)
            x_result = self.forward_propagation(x)
            # choose best class (max of elements)
            maxx = -1
            ind_maxx = None
            for ind, val in enumerate(x_result):
                if val > maxx:
                    maxx = val
                    ind_maxx = ind
            # compare to output
            if y == ind_maxx:
                result += 1
        # return value is how_many_good / how_many_tested
        return result / len(data_inputs)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1.0 - sigmoid(z))

### 3.3. Funkcja walidacyjna
+ Funkcja VALIDATE_NETWORK otrzymuje jako parametry: listę hiperparametrów do sprawdzenia, wartość parametru beta dla spadku gradientu, zbiór testowy i zbiór walidacyjny.
+ Sprawdza ona jak działa dany hiperparametr poprzez wytrenowanie z jego użyciem sieci na zbiorze testowym i przetestowanie na zbiorze walidacyjnym.
+ Funkcja zwraca sieć nauczoną na hiperparametrze, na którym sukces klasyfikacji był największy.

In [4]:
def validate_network(hiperparam_list, SGD_beta, train_set, valid_set):
    best_network = None
    best_score = 0
    best_hiperparam = None
    # for every hiperparam
    for hiperparam in hiperparam_list:
        net = Network(hiperparam)
        net.train_network(train_set, SGD_beta)
        net_score = net.score_network(valid_set)
        print(f"Score for network with hiperparam: {hiperparam} equals {net_score}")
        # choose if best so far
        if net_score > best_score:
            best_score = net_score
            best_network = net
            best_hiperparam = hiperparam
    print(f"Best network is with hiperparam: {best_hiperparam} with score {best_score}")
    return best_network

### 3.4. Funkcja testująca
+ Funkcja wyświetla sukces testu sieci na zbiorze testującym.

In [5]:
def test_network(network, test_set):
    score = network.score_network(test_set)
    print(f"Score for network on test set equals: {score}")

## 4. Test implementacji
### 4.1. Definicja hiperparametrów, walidacja na zbiorze uczącym i walidacyjnym
+ Na początku występuje wydzielenie zbiorów: testowego, walidacyjnego i testowego.
+ Następnie zostaje zdefiniowana lista hiperaparametrów do przebadania.
+ Ustalony zostaje parametr beta dla GD.
+ Na końcu wywoływana jest operacja walidacji sieci.
+ Wynik: zbiór stosunków poprawnych klasyfikacji zbioru walidacyjnego do wszystkich przeprowadzonych klasyfikacji dla każdego hiperparametru (np. 0.9123 = 91,23% sukces).
+ Dodatkowo zostaje przedstawione, który z hiperparametrów daje najlepsza klasyfikację dla zbioru walidacyjnego, wytrenowana na nim sieć jest zapisywana do zmiennej network. 

In [6]:
TRAIN_SET, VALIDATION_SET, TEST_SET = generate_mnist_datasets()
hiperparams_list = [
    [784, 30, 10],
    [784, 100, 80, 55, 30, 10],
    [784, 10, 10],
    [784, 800, 10],
    [784, 300, 300, 10],
    [784, 200, 100, 10],
    [784, 100, 10],
    [784, 300, 10],
    [784, 300, 200, 100, 50, 20, 10],
    [784, 100, 100, 100, 10],
    [784, 1, 10]
]
beta = 0.1
network = validate_network(hiperparams_list, beta, TRAIN_SET, VALIDATION_SET)

Score for network with hiperparam: [784, 30, 10] equals 0.9561
Score for network with hiperparam: [784, 100, 80, 55, 30, 10] equals 0.9135
Score for network with hiperparam: [784, 10, 10] equals 0.9239
Score for network with hiperparam: [784, 800, 10] equals 0.9738
Score for network with hiperparam: [784, 300, 300, 10] equals 0.9686
Score for network with hiperparam: [784, 200, 100, 10] equals 0.9679
Score for network with hiperparam: [784, 100, 10] equals 0.9704
Score for network with hiperparam: [784, 300, 10] equals 0.975
Score for network with hiperparam: [784, 300, 200, 100, 50, 20, 10] equals 0.103
Score for network with hiperparam: [784, 100, 100, 100, 10] equals 0.9564
Score for network with hiperparam: [784, 1, 10] equals 0.2061
Best network is with hiperparam: [784, 300, 10] with score 0.975


+ Dla większości problemów jedna warstwa ukryta w zupełności wystarcza, natomiast musi występować odpowiednia liczba neuronów w tej warstwie.
+ Użycie zbyt małej ilości neuronów w warstwie ukrytej (np. [784, 1, 10]) może doprowadzić do zjawiska zwanego underfitting. Wówczas nasz model jest zbyt prosty, by móc poradzić sobie z danym zadaniem (w przypadku zbioru MNIST model sprawdza czy zestaw pixeli dopasowuje sie do tylko jednego mozliwego ksztaltu - prawdopodobnie za każdym razem, niezależnie od danych wejsciowych wybiera jedną albo dwie liczby, które mu odpowiadają, stąd wynik to 20%, jednak nie ma w tym żadnej użyteczności). Zazwyczaj dobrym rozwiązaniem jest zwiększenie ilości neuronów w danej warstwie.
+ Zjawiskiem przeciwnym do underfitting jest overfitting. Overfitting występuje wtedy, gdy sieć neuronowa ma tak dużą zdolność przetwarzania informacji, że ograniczona ilość informacji zawarta w zestawie treningowym nie wystarcza do wytrenowania wszystkich neuronów w warstwach ukrytych. Taka sytuacja mogła zajść w przykładzie z warstwami [784, 300, 200, 100, 50, 20, 10]. Model stał się tak skomplikowany, że dla zbioru MNIST jako daną liczbę wykrywał tylko tą samą, ale napisaną zaledwie za pomocą niewielu charakterów pisma.
+ Oczywiście dodatkowym problemem ze zbyt dużą ilością warstw ukrytych i liczby neuronów jest czas potrzebny na wytrenowanie, który może, dla naprawdę skomplikowanych sieci, być bardzo długi.
+ Istnieje wiele możliwości doboru odpowiednich parametrów. Dla większości sytuacji w zupełności wystarczy jedna warstwa ukryta. Większy problem jest z doborem liczby neuronów. Przeważnie liczba ta powinna być z przedziału (liczba neuronów warstwy wejściowej; liczba neuronów warstwy wyjściowej), przy czym nie powinna być ona za mała."

### 4.2. Ostateczny test sieci
+ Sieć nauczona na zbiorze trenującym z hiperparametrem wybranym na podstawie zbioru walidacyjnego jest wykorzystywana do przetestowania klasyfikatora na zbiorze testującym.
+ W wyniku otrzymujemy sukces (stosunek poprawnych klasyfikacji do wszystkich klasyfikacji).

In [7]:
test_network(network, TEST_SET)

Score for network on test set equals: 0.9743


+ Jak widać, sieć popełnia błąd przy klasyfikacji 2,57% podanych próbek, co oznacza, że sieć jest bardzo dobrym klasyfikatorem.